# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。

(使用Day023_ETtoday_News_HW)




In [3]:
!pip install selenium

In [5]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
ETtoday_url = 'https://www.ettoday.net/news/focus/%E5%9C%8B%E9%9A%9B/'  #國際新聞

In [6]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1
#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
while True:
     # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break
    last_height = new_height
    #提示：可參考以下的Stack Overflow: 
    #https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    
print("bottom of the web")

bottom of the web


In [19]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
#   for p in news_content:
#        """
#        .string屬性說明：
#        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
#        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
#        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
#        """
#   if ((p.string) is not None):
#            print(p.string)

In [23]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})
news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

In [25]:
import time
start = time.time()
for i, news_item in enumerate(news_block):
    #print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    #print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)
end = time.time()
total = end-start
print(total, ' seconds')

22.019173622131348  seconds


In [27]:
import _thread

start_multi = time.time()
for i, news_item in enumerate(news_block):
    #print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    #print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    _thread.start_new_thread( getNewsDetailContent, (externalLink,) )
    #_thread.start_new_thread ( 函式, (變數們 , <-- 注意 只有一個變數還是要有逗點) )
end_multi = time.time()
total_multi = end_multi-start_multi
print(total_multi, ' seconds')

0.02293848991394043  seconds


In [31]:
print('時間節省', total - total_multi, '秒')
print('速度提升', (total - total_multi)/total_multi , '%')

時間節省 21.996235132217407 秒
速度提升 958.9225452391099 %
